# Support Vector Machine - Predicción de ImpactPlayerScore en CS:GO
Este notebook sigue la metodología CRISP-DM hasta la fase 4 para evaluar el modelo de regresión: **Support Vector Machine**.

In [ ]:
# Cargar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.svm import SVR

In [ ]:
#Cargar datos y crear variables

url = "https://raw.githubusercontent.com/Seba-RiveraC/Crisp_DM_CSGO/refs/heads/master/Anexo%20ET_demo_round_traces_2022%20(1).csv"
df = pd.read_csv(url, sep=';')
# Crear variables necesarias
df['ImpactPlayerScore'] = df['RoundKills'] + 0.5 * df['RoundAssists'] + df['RoundHeadshots']
df['KAST'] = ((df['RoundKills'] > 0) | (df['RoundAssists'] > 0)).astype(int)
df = df.sample(frac=0.5, random_state=42)

<ipython-input-2-453c2a655bc6>:4: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=';')


In [ ]:
# Fase 3 - Preparación de datos y limpieza
df_model = df.copy()
df_model = df_model.applymap(lambda x: str(x) if isinstance(x, str) or isinstance(x, bool) else x)
df_model = df_model.replace({'True': True, 'False': False, 'TRUE': True, 'FALSE': False})
for col in df_model.columns:
    if df_model[col].dtype == bool:
        df_model[col] = df_model[col].astype(int)
categorical_cols = df_model.select_dtypes(include='object').columns.tolist()
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

<ipython-input-3-6e3455b4a4bb>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_model = df_model.applymap(lambda x: str(x) if isinstance(x, str) or isinstance(x, bool) else x)
<ipython-input-3-6e3455b4a4bb>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model = df_model.replace({'True': True, 'False': False, 'TRUE': True, 'FALSE': False})


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Cargar y copiar datos
df_model = df.copy()

def convertir_a_float(valor):
    if isinstance(valor, str):
        valor = valor.replace('.', '').replace(',', '.')
        try:
            return float(valor)
        except:
            return np.nan
    return valor

# Aplicar limpieza
columnas = ['TimeAlive', 'TravelledDistance', 'KAST', 'ImpactPlayerScore']
for col in columnas:
    df_model[col] = df_model[col].apply(convertir_a_float)

# Eliminar nulos
df_model.dropna(subset=columnas, inplace=True)

# Variables predictoras y objetivo
X = df_model[['KAST', 'TimeAlive', 'TravelledDistance']]
y = df_model['ImpactPlayerScore']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

# Modelo SVM con kernel RBF
modelo_svm = SVR(kernel='rbf')
modelo_svm.fit(X_train, y_train)
y_pred = modelo_svm.predict(X_test)

# Evaluación
print("\nModelo: SVM (con escalado)")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))




Modelo: SVM (con escalado)
MAE: 0.5133208205863873
MSE: 0.8485317531462443
RMSE: 0.9211578329180317
R²: 0.5355842745726468


## Analisis
Al ejecutar el svm sin escalados los resultados eran muy difusos, pero al aplicar escalado el modelo rinde muy bien, con bajos margenes de error y con r2 alto, al punto de ser el modelo que mejor funciona de los 5.
En parte concluyendo hemos decidido usar el modelo de regresion de SVM para los futuros avances en la puesta en marcha del proyecto para las siguientes entregas